# **Mount**

Mount the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install import-ipynb
%cd "/content/gdrive/MyDrive/progetto/code/"

Mounted at /content/gdrive
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=0b838945354fcba8a83a5f3dec7b60014d18692e222b9e69dfc0a8b8f6862000
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/gdrive/.shortcut-targets-by-id/1IrDGYaOd_7sICD_EFnZIaiOAtwpfZeii/progetto/code


# **Import**

Import the LSH_Index file,necessary to import the index

In [ ]:
import import_ipynb
import pickle
import numpy as np
from LSH_Index import *

importing Jupyter notebook from LSH_Index.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Pickles**

Import the pickles,previously saved,belonging to indexes and the testing dataset. (commented lines belong to various tries)

In [ ]:

pick_path = '/content/gdrive/MyDrive/progetto/pickles/lsh_models'
#test_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_features_finetuning.npy"
#label_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_labels_finetuning.npy"
#test_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_features.npy"
#label_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_labels.npy"

#testo con il fune-tuning early stop
test_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_features_finetuning_early_stop.npy"
label_path = "/content/gdrive/MyDrive/progetto/pickles/testing_image_labels_finetuning_early_stop.npy"


cosine = "/cosine_similarity/"
eucl = "/euclidean_distance/"

pickle_name = "complete(70,8)_early.pickle"

print("Reading pickle..." )
LSHcos = pickle.load( open( pick_path + cosine + pickle_name, "rb" ) )
LSHeuc = pickle.load( open( pick_path + eucl + pickle_name, "rb" ) )
print("pickle loaded.")

print("Loading features and labels...")
test_features = np.load(  test_path  )
test_labels = np.load( label_path )
print("features loaded.")

Reading pickle...
pickle loaded.
Loading features and labels...
features loaded.


# **Compute AP at K**

## Average precision at K 

$AP@k= \frac{1}{N(k)} \sum_{i=1}^{k} \frac{TPseen(i)}{i}$


$TP = true positives$

$N(k) = min(k,TPtotal)$

*TPseen(i) = {0 if i°=FALSE; 1 if i°=TRUE}*

## Mean Average Precision at K

The mAP@k is the mean of AP@K obtained from different queries

$mAP@k= \frac{1}{N} \sum_{i=1}^{N} AP@k_i$

In [ ]:
#compute AP@k
def compute_ap_at_k(ranked_relevance, k):
  #ranked_relevance is a boolean array of dim k. true if retrieved element is relevant for the query, false if not relevant 
  total_relevant_items = ranked_relevance.sum()

  # Precision@i for each query, i.e. the number of relevant items @i divided by i
  #cumsum [1,3,2]=>[1,4,6]
  prec_at_i = ranked_relevance.cumsum() / np.arange(1, k + 1) 

  # sum of prec_at_i only for relevant, divide for total number of relevant items 
  aps =  (ranked_relevance * prec_at_i).sum() / ( total_relevant_items + np.finfo(float).eps ) 
  #print(aps)

  return aps

def retrieve_query_label(results):
  label_results = []
  for result in results:
    label_results.append(result[1])
  return label_results

# **Compute APs**

For all the features belonging to the testing dataset, we retrieve using the index the 10 most similar images. Of this 10 images we retrieve their labels, and we use them in order to compute the ap; the ap is then appended in an array in the position corresponding to the label of the searched feature. In addition we save the time needed to perform each query. The test are performed for the indexes constructed using either the cosine or euclideans search method. (commented lines belong to various tries)

In [ ]:
import time

#ap_cos = [[] for _ in range(25)]
#ap_eucl = [[] for _ in range(25)]
#time_eucl = [[] for _ in range(25)]
#time_cos = [[] for _ in range(25)]

ap_cos = [[] for _ in range(25)]
ap_eucl = [[] for _ in range(25)]
time_eucl = [[] for _ in range(25)]
time_cos = [[] for _ in range(25)]
print("Start testing...")
index = 0

for i in range(len(test_features)):
  index = 0
  #print(i)
  #for g in [4, 8, 16, 32]:
  #for g in [2,3,4,5,6,7,8]:
  for g in [30,40,50,60,70]:
  #for g in [40, 45, 50, 55, 60, 65, 70]:
  #for g in [70, 75, 80, 85, 90, 95, 100]:
    #for h in [4, 8, 16, 32]:
    #for h in [2,3,4,5,6,7,8]:
    for h in [2,3,4,6,8]:
    #for h in [5,6,7]:
    #for h in [7,8]:
      startTimeCos = time.perf_counter()
      resultsCos = LSHcos.search( test_features[i], g, h,10)
      endTimeCos = time.perf_counter()
      #print("features: "+str(i)+" g:"+str(g)+" h: "+str(h)+" cosine "+str(endTimeCos-startTimeCos))
      time_cos[index].append(endTimeCos-startTimeCos)
      startTimeEucl = time.perf_counter()
      resultsEucl = LSHeuc.search( test_features[i], g, h,10)
      endTimeEucl = time.perf_counter()
      #print("features: "+str(i)+" g:"+str(g)+" h: "+str(h)+" eucl "+str(endTimeEucl-startTimeEucl))
      time_eucl[index].append(endTimeEucl-startTimeEucl)
      labels_resultCos = retrieve_query_label( resultsCos )
      labels_resultEucl = retrieve_query_label( resultsEucl )

      ranked_relevance_cos = []
      for label in labels_resultCos:
        if test_labels[i] == label:
          res = 1
        else:
          res = 0

        ranked_relevance_cos.append(res)

      ranked_relevance_eucl = []
      for label in labels_resultEucl:
        if test_labels[i] == label:
          res = 1
        else:
          res = 0

        ranked_relevance_eucl.append(res)

      ranked_relevance_eucl = np.array(ranked_relevance_eucl)
      ranked_relevance_cos = np.array(ranked_relevance_cos)

      ap_k_cos = compute_ap_at_k(ranked_relevance_cos, len(ranked_relevance_cos) ) 
      ap_cos[index].append(ap_k_cos)  

      ap_k_eucl = compute_ap_at_k(ranked_relevance_eucl, len(ranked_relevance_eucl) ) 
      ap_eucl[index].append(ap_k_eucl)
      index += 1 

print("Testing finished.")


# **Save raw performance**

We save as pickles the aps and average search times

In [ ]:
import numpy as np
print("Saving results...")

ap_cos = np.array(ap_cos)
ap_eucl = np.array(ap_eucl)
time_cos = np.array(time_cos)
time_eucl = np.array(time_eucl)

np.save("/content/gdrive/MyDrive/progetto/results/cosine/ap_matrix_k=10,g=70,h=8_2.npy", ap_cos)
np.save("/content/gdrive/MyDrive/progetto/results/eucl/ap_matrix_k=10,g=70,h=8_2.npy", ap_eucl)
np.save("/content/gdrive/MyDrive/progetto/results/cosine/time_matrix_k=10,g=70,h=8_2.npy", time_cos)
np.save("/content/gdrive/MyDrive/progetto/results/eucl/time_matrix_k=10,g=70,h=8_2.npy", time_eucl)
print("Results saved.")


Saving results...
Results saved.


# **Save processed performance**

We save the processed performance,namely mean average precision and average search time, as pickles

In [ ]:
map_cos = []
map_eucl = []
averageTime_cos = []
averageTime_eucl = []

print("Evaluating map...")
for index in range(25):  
  ap_cos_k = ap_cos[index]
  map_cos.append( np.mean( ap_cos_k ) )
  time_cos_k = time_cos[index]
  averageTime_cos.append(np.mean(time_cos_k))

  ap_eucl_k = ap_eucl[index]
  map_eucl.append( np.mean( ap_eucl_k ) )
  time_eucl_k = time_eucl[index]
  averageTime_eucl.append(np.mean(time_eucl_k))
print("map evaluated...")

print("Saving results...")

map_cos = np.array(map_cos)
map_eucl = np.array(map_eucl)
averageTime_cos = np.array(averageTime_cos)
averageTime_eucl = np.array(averageTime_eucl)

np.save("/content/gdrive/MyDrive/progetto/results/cosine/map_vector_k=10,g=70,h=8_2.npy", map_cos)
np.save("/content/gdrive/MyDrive/progetto/results/eucl/map_vector_k=10,g=70,h=8_2.npy", map_eucl)
np.save("/content/gdrive/MyDrive/progetto/results/cosine/mean_time_k=10,g=70,h=8_2.npy", averageTime_cos)
np.save("/content/gdrive/MyDrive/progetto/results/eucl/mean_time_k=10,g=70,h=8_2.npy", averageTime_eucl)
print("Results saved.")



Evaluating map...
map evaluated...
Saving results...
Results saved.


# **Print processed performance**

(commented lines belong to various tries)

In [ ]:
#
# u can open the file map_vector.npy
#

print("Printing the ap matrix...")
print(ap_cos)
print(ap_eucl)

print("Printing the map vectors...")
print(map_cos)
print(map_eucl)

print("Printing results...")

index = 0  
#for g in [4, 8, 16, 32]:
#for g in [2,3,4,5,6,7,8]: 
for g in [30,40,50,60,70]:
#for g in [40, 45, 50, 55, 60, 65, 70]:
#for g in [70, 75, 80, 85, 90, 95, 100]:
  #for h in [4, 8, 16, 32]:
  #for h in [2,3,4,5,6,7,8]:
  for h in [2,3,4,6,8]:
  #for h in [5,6,7]:
  #for h in [7,8]:
    print("cosine MAP g=" + str(g) + " h=" + str(h) + ": " + str( map_cos[index ]))
    print("cosine search g=" + str(g) + " h=" + str(h) + ": " + str( averageTime_cos[index ]))
    print("euclidean MAP g=" + str(g) + " h=" + str(h) + ": " + str( map_eucl[index ]))
    print("euclidean search g=" + str(g) + " h=" + str(h) + ": " + str( averageTime_eucl[index ]))
    print("\n")

    index +=1 
    

Printing the ap matrix...
[[0.         0.         0.         ... 0.59404762 1.         0.98888889]
 [0.         0.         0.         ... 0.86111111 1.         0.96265432]
 [0.         0.         0.         ... 0.46928571 0.93253968 1.        ]
 ...
 [0.         0.         0.33333333 ... 0.1        1.         1.        ]
 [0.         0.         1.         ... 0.90935374 0.4768254  0.81632653]
 [0.         0.         0.         ... 0.53333333 0.925      1.        ]]
[[0.         0.         0.33333333 ... 0.         0.96265432 1.        ]
 [0.16666667 0.1        0.         ... 0.44428571 0.92826279 0.97654321]
 [0.25       0.         0.         ... 0.52162698 0.81051587 1.        ]
 ...
 [0.         0.         0.         ... 0.50218254 0.97619048 1.        ]
 [0.         0.         0.         ... 0.51481481 0.70925926 0.96265432]
 [0.         0.41666667 0.325      ... 0.41666667 0.5        0.94285714]]
Printing the map vectors...
[0.64134919 0.62833508 0.61723895 0.58837446 0.44172769 0.